# Upload png stimuli to s3

In [17]:
import os
from glob import glob
import boto3
import botocore
from IPython.display import clear_output
import json
import pandas as pd
from PIL import Image

In [18]:
def list_files(path, ext='png'):
    result = [y for x in os.walk(path) for y in glob(os.path.join(x[0], '*.%s' % ext))]
    return result

In [19]:
## helper to speed things up by not uploading images if they already exist, can be overriden 
def check_exists(s3, bucket_name, stim_name):
    try:
        s3.Object(bucket_name,stim_name).load()    
        return True
    except botocore.exceptions.ClientError as e:    
        if (e.response['Error']['Code'] == "404"):
            print('The object does not exist.')
            return False
        else:
            print('Something else has gone wrong with {}'.format(stim_name))

In [20]:
curiotower_testing_data = {'bucket_name': 'curiotower',
                          'path_to_stim': 'images',
                          'full_stim_paths': '---------',  # use list_files(path_to_stim)
                          'stim_name': '-----------'} # use os.path.split(path_to_file)[-1]

In [23]:
## set up paths, etc.
bucket_name = 'curiotower-tdw' ## which S3 bucket to upload to 
path_to_stim = 'tdw_png'
full_stim_paths = [x for x in list_files('./tdw_png') if x !='.DS_Store']
print('We have {} images to upload.'.format(len(full_stim_paths)))

We have 146 images to upload.


In [24]:
reallyRun = 1
if reallyRun: 

    ## establish connection to s3 
    s3 = boto3.resource('s3')

    ## create a bucket with the appropriate bucket name
    try: 
        b = s3.create_bucket(Bucket=bucket_name) 
        print('Created new bucket.')
    except:
        b = s3.Bucket(bucket_name)
        print('Bucket already exists.')

    ## do we want to overwrite files on s3?
    overwrite = False
    
    ## set bucket and objects to public
    b.Acl().put(ACL='public-read') ## sets bucket to public

    ## now let's loop through stim paths and actually upload to s3 (woot!)
    for i,path_to_file in enumerate(full_stim_paths):        # use sorted(full_stim_paths) when not using photodraw32
        stim_name = path_to_file.split('/')[-1]      # default: os.path.split(path_to_file)[-1]
        if ((check_exists(s3, bucket_name, stim_name)==False) | (overwrite==True)):
            print('Now uploading {} as {} | {} of {}'.format(os.path.split(path_to_file)[-1],stim_name,(i+1),len(full_stim_paths)))
            s3.Object(bucket_name,stim_name).put(Body=open(path_to_file,'rb')) ## upload stimuli
            s3.Object(bucket_name,stim_name).Acl().put(ACL='public-read') ## set access controls
        else: 
            print('Skipping {} | {} of {} because it already exists.'.format(os.path.split(path_to_file)[-1],(i+1),len(full_stim_paths)))
        clear_output(wait=True)
print('Done!')

Done!


In [10]:
for my_bucket_object in b.objects.all():
    print(my_bucket_object)

s3.ObjectSummary(bucket_name='curiotower-tdw', key='curiotower_2_high_0000_0.png')
s3.ObjectSummary(bucket_name='curiotower-tdw', key='curiotower_2_high_0000_1.png')
s3.ObjectSummary(bucket_name='curiotower-tdw', key='curiotower_2_high_0001_0.png')
s3.ObjectSummary(bucket_name='curiotower-tdw', key='curiotower_2_high_0001_1.png')
s3.ObjectSummary(bucket_name='curiotower-tdw', key='curiotower_2_high_0002_0.png')
s3.ObjectSummary(bucket_name='curiotower-tdw', key='curiotower_2_high_0002_1.png')
s3.ObjectSummary(bucket_name='curiotower-tdw', key='curiotower_2_high_0003_0.png')
s3.ObjectSummary(bucket_name='curiotower-tdw', key='curiotower_2_high_0003_1.png')
s3.ObjectSummary(bucket_name='curiotower-tdw', key='curiotower_2_high_0004_0.png')
s3.ObjectSummary(bucket_name='curiotower-tdw', key='curiotower_2_high_0004_1.png')
s3.ObjectSummary(bucket_name='curiotower-tdw', key='curiotower_2_high_0005_0.png')
s3.ObjectSummary(bucket_name='curiotower-tdw', key='curiotower_2_high_0005_1.png')
s3.O